In [18]:
#Auotograd
import torch

In [19]:
x=torch.ones(2,2,requires_grad=True)
print(x)


tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [20]:
y=x+2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [21]:
#y是一次运算的结果，所以是有grad_fn的
print(y.grad_fn)

In [22]:
z=y*y*3
out=z.mean()
print(z,out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


In [23]:
a=torch.rand(2,2)
a=((a*3)/(a-1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b=(a*a).sum()
print(b.grad_fn)


False
True


In [24]:
out.backward()
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


In [25]:
x=torch.randn(3,requires_grad=True)
y=x*2
while y.data.norm()<1000:
    y=y*2
print(y)

tensor([-744.3121,  -19.7864,  940.1359], grad_fn=<MulBackward0>)


In [26]:
v=torch.tensor([0.1,1.0,0.0001],dtype=torch.float)
y.backward(v)

print(x.grad)

tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


In [27]:
#卷积，填充
import torch
from torch import nn
#计算卷积层函数，对输入和输出进行升维，降维
def comp_conv2d(conv2d,X):
    X=X.view((1,1)+X.shape)#(1,1)代表批量大小和通道数
    Y=conv2d(X)
    return Y.view(Y.shape[2:])#不关心前两维，批量与通道
conv2d=nn.Conv2d(in_channels=1,out_channels=1,kernel_size=3,padding=1)#注意这个是在两侧分别填充一行/列，所以总共是填充了2，填出10x10
X=torch.rand(8,8)
comp_conv2d(conv2d,X).shape#10 10用3 3卷积，得到8 8

torch.Size([8, 8])

In [28]:
#当然宽和高不同也可以设置不同的填充数使输入输出有相同的宽和高
conv2d=nn.Conv2d(in_channels=1,out_channels=1,kernel_size=(5,3),padding=(2,1))#高为5宽为3的卷积核，填充是2，1
comp_conv2d(conv2d,X).shape


torch.Size([8, 8])

In [29]:
#设置步幅
conv2d=nn.Conv2d(1,1,kernel_size=3,padding=1,stride=2)
comp_conv2d(conv2d,X).shape

torch.Size([4, 4])

In [30]:
#尝试池化层
X=torch.arange(16,dtype=torch.float).view((1,1,4,4))
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

In [31]:
pool2d=nn.MaxPool2d(3)
pool2d(X)#默认步幅形状是(3,3)

tensor([[[[10.]]]])

In [32]:
pool2d=nn.MaxPool2d(3,padding=1,stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

In [33]:
#define the network
import torch.nn as nn
import torch.nn.functional as F
import torch
class Net(nn.Module):

    def __init__(self):
        super(Net,self).__init__()
        #一个输入通道，6个输出通道，3x3卷积核
        #kernel1
        self.conv1=nn.Conv2d(1,6,3)#对应的是输入，输出通道，核大小
        self.conv2=nn.Conv2d(6,16,3)
        #affine计算
        self.fc1=nn.Linear(16*6*6,120)
        self.fc2=nn.Linear(120,84)
        self.fc3=nn.Linear(84,10)
    def forward(self,x):
        #Max pooling over a (2,2) window
        x=F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x=F.max_pool2d(F.relu(self.conv2(x)),(2,2))
        x=x.view(-1,self.num_flat_features(x))
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return x
    def num_flat_features(self,x):
        size=x.size()[1:]#包含除了batchsize之外的维度
        num_features=1
        for s in size:
            num_features*=s
        return num_features
net=Net()
print(net)
#我们只需要定义前馈函数，反向传播函数会用自动求导求出

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [34]:
#可以洗参数可以由net.parameters()获取
params=list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 3, 3])


In [35]:
#尝试一下输入32x32.注意，这个网络是LeNet,输入大小要是32x32
input=torch.randn(1,1,32,32)
out=net(input)
print(out)

tensor([[ 2.3007e-02,  4.6504e-02, -3.1089e-02,  1.0999e-01,  4.8303e-02,
          3.5057e-02, -1.3241e-01,  4.7866e-05, -1.3791e-01,  6.6794e-02]],
       grad_fn=<AddmmBackward>)


In [36]:
net.zero_grad()
out.backward(torch.randn(1,10))
#注意torch.nn只支持mini-batches的输入，比如nn.Conv2d会接收一个四维的张量，样本容量x通道数x高x宽。如果只有一个样本，使用input.unsqueeze(0)增加一个维度


In [37]:
#计算损失
output=net(input)
target=torch.randn(10)
target=target.view(1,-1)
criterion=nn.MSELoss()

loss=criterion(output,target)
print(loss)

tensor(1.0245, grad_fn=<MseLossBackward>)


In [38]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

In [40]:
#反向传播。我们要先清零梯度
net.zero_grad()

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0056,  0.0046, -0.0114,  0.0121,  0.0134,  0.0100])


In [41]:
#更新梯度
learning_rate=0.01
for f in net.parameters():
    f.data.sub_(f.grad.data*learning_rate)

In [43]:
#若想使用更多的梯度更新方法，我们可以使用torch.optim
import torch.optim as optim
import time
start=time.time()
optimizer=optim.SGD(net.parameters(),lr=0.01)

optimizer.zero_grad()#梯度置为零
output=net(input)
loss=criterion(output,target)
loss.backward()
optimizer.step()#进行更新

print(time.time()-start)

0.005984783172607422
